In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
!pip install transformers==4.24.0

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [3]:
!pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [4]:
import torch
import transformers
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime
import warnings

#Dataset Imports
import csv
from IPython.display import Audio, display
import torchaudio
from transformers import Wav2Vec2FeatureExtractor
from torch.nn.utils.rnn import pad_sequence

warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [5]:
print(transformers.__version__)

4.24.0


## Dataloaders

In [6]:
# {'A':1, 'S':2, 'H':3, 'U':4, 'F':5, 'D':6, 'C':7, 'N':8, 'O':9}
EMOMAP = {'A':0, 'S':1, 'H':2,'D':3,'N':4}

In [7]:
NSC_ASR_Root = '/home/ubuntu/data/NSC/'
NSC_test_dataset = '/home/ubuntu/data/NSC/NSC_part5_labelled_emotion/'

In [8]:
from transformers import Wav2Vec2Processor
model_name_or_path = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path,)
target_sampling_rate = processor.feature_extractor.sampling_rate
print(target_sampling_rate)

16000


In [9]:
class NSCSpeechDatasetASR(torch.utils.data.Dataset):
    
    def __init__(self): 
        
        self.audio_dir = NSC_ASR_Root + 'top20k/'
        self.labels_dir = NSC_ASR_Root + 'text'
        self.audio = []
        self.labels = []
        self.audio = sorted(os.listdir(self.audio_dir)[0:100])

        #Get the correct labels for the 20000 that we have.
        with open(self.labels_dir) as f:
            lines = f.readlines()
            start = False
            for l in lines:
                idx = int(l[4:8])
                #Start at APP_4001 and take 20000 from there
                if idx >= 4001 and len(self.labels) != len(self.audio):
                    #Remove new line and extract transcript
                    self.labels.append(l[:-1].split(" ", 1))
        print(len(self.audio))
        print(len(self.labels))
        assert(len(self.audio) == len(self.labels))
        self.length = len(self.audio)
        #Sanity Check!
        #Could be commented out..
        # for i in range(len(self.audio)):
        #     if(self.audio[i][:-4] != self.labels[i][0]):
        #         print(self.audio[i])
        #         print(self.labels[i][0])
        #         break
                
        self.labels = [x[1] for x in self.labels][0:100]

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        audio = self.audio[ind]
        #Sanity Check
        print(audio)
        label = self.labels[ind]
        #load audio when getting the item. If we do it in init, computer blue screens.
        waveform, sample_rate = torchaudio.load(self.audio_dir + audio)
        waveform = processor(waveform, sampling_rate = 16000,padding=True, return_tensors="pt", device = device)
        # waveform = waveform.to(device)
        # label = label.to(device)
        waveform['labels'] = label

        return waveform

In [10]:
class NSCDatasetEmotion(torch.utils.data.Dataset):
    
    def __init__(self): 
        
        audio_dir = NSC_test_dataset
        #quick way of looping subdirectories. Dataset only has 4 categories. 
        subdirectory = [('Anger/', 0), ('Sad/', 1), ('Happy/', 2), ('Neutral/', 4)]
        self.audio = []
        self.labels = []
        for sub, label in subdirectory:
            print(audio_dir + sub)
            NSCaudios = os.listdir(audio_dir + sub)
            self.audio += [audio_dir + sub + x for x in NSCaudios[0:20]]
            self.labels += [label]*len(NSCaudios[0:20]) 
        #Sanitycheck1
        assert(len(self.audio) == len(self.labels))
        self.length = len(self.audio)
        
        #self.audio = 
        
    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        audio = self.audio[ind]
        label = self.labels[ind]
        #load audio when getting the item. If we do it in init, computer blue screens.
        waveform, sample_rate = torchaudio.load(audio)
        waveform = processor(waveform, sampling_rate = 16000,padding=True, return_tensors="pt", device = device)
        # waveform = waveform.to(device)
        # label = label.to(device)
        waveform['labels'] = label

        return waveform
        #return waveform, label

In [11]:
NSCTest = NSCDatasetEmotion()
NSCTest[0]
# NSCtest_dataset = Dataset.from_list(NSCTest)

# #stratified sort to train/test splits. Requires encoding the columns to classes first.
# NSCtest_dataset2 = NSCtest_dataset.class_encode_column('labels')
# #NSCtest_dataset2.train_test_split(test_size = 0.1, stratify_by_column = 'labels')

/home/ubuntu/data/NSC/NSC_part5_labelled_emotion/Anger/
/home/ubuntu/data/NSC/NSC_part5_labelled_emotion/Sad/
/home/ubuntu/data/NSC/NSC_part5_labelled_emotion/Happy/
/home/ubuntu/data/NSC/NSC_part5_labelled_emotion/Neutral/


{'input_values': tensor([[[0.2524, 0.3185, 0.3613,  ..., 0.2399, 0.2616, 0.2622]]]), 'labels': 0}

In [12]:
NSCSpeech = NSCSpeechDatasetASR()
for i in range(5):
    print(NSCTest[i])

100
100
{'input_values': tensor([[[0.2524, 0.3185, 0.3613,  ..., 0.2399, 0.2616, 0.2622]]]), 'labels': 0}
{'input_values': tensor([[[-0.0588, -0.0411, -0.0239,  ...,  0.0412,  0.0393,  0.0015]]]), 'labels': 0}
{'input_values': tensor([[[-0.1709, -0.1947, -0.1973,  ..., -0.0908,  0.0167,  0.0577]]]), 'labels': 0}
{'input_values': tensor([[[-0.0713, -0.3233, -0.4515,  ...,  1.1354,  1.3906,  1.7724]]]), 'labels': 0}
{'input_values': tensor([[[ 0.0322,  0.0168, -0.0074,  ...,  1.2563,  1.2658,  1.2795]]]), 'labels': 0}


In [13]:
from datasets import load_dataset
MSP_podcast_train = load_dataset("dlproject/msp_train_hubert")
MSP_podcast_val = load_dataset("dlproject/msp_val_hubert")

Using custom data configuration dlproject--msp_train_hubert-d9cb93d944f5240e
Found cached dataset parquet (/home/ubuntu/.cache/huggingface/datasets/dlproject___parquet/dlproject--msp_train_hubert-d9cb93d944f5240e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  5.85it/s]
Using custom data configuration dlproject--msp_val_hubert-89eed54aad19cdf2
Found cached dataset parquet (/home/ubuntu/.cache/huggingface/datasets/dlproject___parquet/dlproject--msp_val_hubert-89eed54aad19cdf2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 38.60it/s]


In [14]:
NSCSpeech.length

100

In [15]:
'''
class CombinedDataForDomainAdaptation(torch.utils.data.Dataset):
    
    def __init__(self): 
        self.msp_data = MSP_podcast_train
        self.nsc_data = NSCSpeech
        self.length = MSP_podcast_train['train'].num_rows + NSCSpeech.length
        
        all_audio_records = []
        for audio_record in self.msp_data['train']:
            audio_record['src'] = "eng"
            all_audio_records.append(audio_record)
        
        for audio_record in self.nsc_data:
            audio_record['src'] = "sing_eng"
            all_audio_records.append(audio_record)
        
        self.audio = all_audio_records
            
        
    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        audio = self.audio[ind]['input_values']
            
        label = self.audio[ind]['labels']
        return audio, label
        #return waveform, label
        
    def collate_fn(self, batch):
        
        for record in batch:
            if record['src'] == 'eng':
        batch_audio = [x[0].reshape(-1) for x in batch]
'''

def create_combined_batches(batch_size):
    #num_batches = (MSP_podcast_train['train'].num_rows + NSCSpeech.length)//batch_size
    num_batches = (100 + NSCSpeech.length)//batch_size
    combined_batches = []
    for i in range(num_batches):
        batch = []
        #batch.append(MSP_podcast_train['train'][0:batch_size//2])
        batch_input_values = []
        batch_labels = []
        for record in range(batch_size//2):
            batch_input_values.append(torch.tensor(MSP_podcast_train['train'][record]['input_values']))
            batch_labels.append(MSP_podcast_train['train']['labels'])
            batch.append({'input_values':batch_input_values, 'labels':batch_labels})
        for record in range(batch_size//2):
            batch_input_values.append(NSCSpeech[record]['input_values'])
            batch_labels.append(NSCSpeech[record]['labels'])
            batch.append({'input_values':batch_input_values, 'labels':batch_labels})
        combined_batches.append(batch)
    return combined_batches

In [16]:
combined_batches = create_combined_batches(8)
#train_dataset = load_dataset("dlproject/msp_train_hubert")

app_4001_6001_phnd_deb-1-0783461-0786319.wav
app_4001_6001_phnd_deb-1-0783461-0786319.wav
app_4004_6007_phnd_deb-1-0900535-0906250.wav
app_4004_6007_phnd_deb-1-0900535-0906250.wav
app_4004_6008_phnd_deb-2-0061113-0066908.wav
app_4004_6008_phnd_deb-2-0061113-0066908.wav
app_4004_6008_phnd_deb-2-1156980-1166245.wav
app_4004_6008_phnd_deb-2-1156980-1166245.wav
app_4001_6001_phnd_deb-1-0783461-0786319.wav
app_4001_6001_phnd_deb-1-0783461-0786319.wav
app_4004_6007_phnd_deb-1-0900535-0906250.wav
app_4004_6007_phnd_deb-1-0900535-0906250.wav
app_4004_6008_phnd_deb-2-0061113-0066908.wav
app_4004_6008_phnd_deb-2-0061113-0066908.wav
app_4004_6008_phnd_deb-2-1156980-1166245.wav
app_4004_6008_phnd_deb-2-1156980-1166245.wav
app_4001_6001_phnd_deb-1-0783461-0786319.wav
app_4001_6001_phnd_deb-1-0783461-0786319.wav
app_4004_6007_phnd_deb-1-0900535-0906250.wav
app_4004_6007_phnd_deb-1-0900535-0906250.wav
app_4004_6008_phnd_deb-2-0061113-0066908.wav
app_4004_6008_phnd_deb-2-0061113-0066908.wav
app_4004_6

In [18]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch
import torch.nn.functional as F
import transformers
from transformers import Wav2Vec2Processor
from torch.nn.utils.rnn import pad_sequence


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # all_features = [torch.tensor(feature['input_values']) for feature in features]
        #print(typeall_features)
        #padded_inputs = pad_sequence(all_features)
        #input_features = [{"input_values": torch.tensor(feature["input_values"][0][0]), "attention_mask":torch.tensor(feature["attention_mask"][0])} for feature in features]
        input_features = [{"input_values": feature["input_values"][0][0]} for feature in features]

        label_features = [feature["labels"] for feature in features]

        #input_lengths = [len(x['input_values']) for x in input_features]
        # print(input_lengths)

        #print(input_features)

        #input_features = input_features.to(device)
        #label_features = label_features.to(device)        
        #print(input_features[0]['input_values'].device)
    
        #d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        # batch = {}
        # batch['input_values'] = torch.stack([features['input_values'] for features in input_features])
        # batch['attention_mask'] = torch.stack([features['attention_mask'] for features in input_features])
        
        batch['input_values'] = batch['input_values'].to(device)

        batch["labels"] = torch.tensor(label_features,device=device)

        # print(batch['labels'])
        # print(batch['input_values'].device)
        # print(batch['labels'].device)

        assert len(batch['input_values'])==len(batch["labels"])

        return batch

In [19]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [20]:
import numpy as np
from transformers import EvalPrediction
from torchmetrics import F1Score

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)

    f1 = F1Score(num_classes=5,average='weighted')
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item(), "f1": f1(torch.tensor(preds), torch.tensor(p.label_ids))}

In [21]:
# config
from transformers import AutoConfig, Wav2Vec2Processor
pooling_mode = "mean"
model_name_or_path = "facebook/hubert-base-ls960"
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=len(EMOMAP),
    label2id=EMOMAP,
    id2label={i: label for label,i in EMOMAP.items()},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)


In [22]:
from transformers import HubertModel, HubertForCTC,HubertForSequenceClassification
feature_model = HubertModel.from_pretrained(
    "facebook/hubert-base-ls960",
    config=config,
    ignore_mismatched_sizes=True,
)

emotion_classifier_model = HubertForSequenceClassification.from_pretrained(
    "facebook/hubert-base-ls960",
    config=config,
    ignore_mismatched_sizes=True,
)
emotion_classifier_model.freeze_feature_extractor()

ctc_model_for_asr = HubertForCTC.from_pretrained(
    "facebook/hubert-base-ls960",
     config=config,
     ignore_mismatched_sizes=True,
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['projector.bias', 'projector.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of HubertForCTC were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
from typing import Any, Dict, Union

import torch
from packaging import version
from torch import nn

from transformers import (
    Trainer,
    is_apex_available,
)

if is_apex_available():
    from apex import amp

if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast


class CTCTrainer(Trainer):
    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        """
        Perform a training step on a batch of inputs.

        Subclass and override to inject custom behavior.

        Args:
            model (:obj:`nn.Module`):
                The model to train.
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.

                The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
                argument :obj:`labels`. Check your model's documentation for all accepted arguments.

        Return:
            :obj:`torch.Tensor`: The tensor with training loss on this batch.
        """

        #model = model.to(device)
        model.train()
        inputs = self._prepare_inputs(inputs)

        if self.use_cuda_amp:
            with autocast():
                loss = self.compute_loss(model, inputs)
        else:
            loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        if self.use_cuda_amp:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            self.deepspeed.backward(loss)
        else:
            loss.backward()

        return loss.detach()


In [24]:
from torch.autograd import Function


class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

In [25]:
import torch.nn as nn
# from functions import ReverseLayerF


class DomainAdaptationModelForAccentedSpeech(nn.Module):

    def __init__(self):
        super(DomainAdaptationModelForAccentedSpeech, self).__init__()
        
        # Feature Extractor Model
        self.feature = feature_model
        
        # Emotion Classification Model
        self.class_classifier = emotion_classifier_model
        
        # ASR Model
        self.asr_model = ctc_model_for_asr

        self.domain_classifier = nn.Sequential()
        self.domain_classifier.add_module('d_fc1', nn.Linear(50 * 4 * 4, 100))
        self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(100))
        self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
        self.domain_classifier.add_module('d_fc2', nn.Linear(100, 2))
        self.domain_classifier.add_module('d_softmax', nn.LogSoftmax(dim=1))

    def forward(self, input_data, alpha):
        #input_data = input_data.expand(input_data.data.shape[0], 3, 28, 28)
        feature = self.feature(input_data)
        #feature = feature.view(-1, 50 * 4 * 4)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.class_classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)

        return class_output, domain_output

In [26]:
DANN = DomainAdaptationModelForAccentedSpeech()

In [ ]:
from torch.utils.data import DataLoader
def get_batch_iterator():
    source_loader = MSP_podcast_train

    source_loader = DataLoader(source_loader['train'], batch_size=1)
    # print("SOURCE:")
    # for i,d in enumerate(source_loader):
    #     print(d)
    #     if i>2:
    #         break
            
            
    #print(source_loader)
    target_loader = NSCSpeech
    
    # print("TARGET:")
    # for i,d in enumerate(target_loader):
    #     print(d)
    #     if i>2:
    #         break
   # print(target_loader)
  
    batches = zip(source_loader, target_loader)
    n_batches = min(len(source_loader), len(target_loader))

    print(list(batches))
    return batches, n_batches

In [ ]:
def train(batches, n_batches, model, discriminator, optimizer, criterion, device):
    model.train()

    total_domain_loss = 0
    domain_acc = 0
    total_label_loss = 0
    label_acc = 0
    #print(batches)
    for i, train_data in enumerate(batches):
        print(train_data)
        (source_x_v, source_x_a,source_x_l, source_y_a), \
        (target_x_v, target_x_a, target_x_l) = train_data

        source_x_v = source_x_v.to(device)
        source_x_a = source_x_a.to(device)
        source_x_l = source_x_l.to(device)
        source_y_a = source_y_a.to(device)

        print(source_x_v)
        print(source_x_a)
        
        print(source_x_l)
        print(source_y_a)
        target_x_v = target_x_v.to(device)
        target_x_a = target_x_a.to(device)
        target_x_l = target_x_l.to(device)

        source_encoded_x = model.encoder(source_x_v, source_x_a, source_x_l)
        target_encoded_x = model.encoder(target_x_v, target_x_a, target_x_l)

        encoded_x = torch.cat([source_encoded_x, target_encoded_x])
        encoded_x = encoded_x.to(device)

        domain_y = torch.cat([	torch.ones(source_encoded_x.shape[0], dtype=torch.int64),
                            torch.zeros(target_encoded_x.shape[0], dtype=torch.int64)]).to(device)
        label_y = source_y_a.to(device)

        domain_preds = discriminator(encoded_x).squeeze()
        label_preds = model.recognizer(source_encoded_x)

        domain_loss = criterion(domain_preds, domain_y)
        label_loss = criterion(label_preds, label_y)
        loss = domain_loss + label_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_domain_loss += domain_loss.item()
        total_label_loss += label_loss.item()

        domain_preds = domain_preds.argmax(dim=1, keepdim=True)
        domain_acc += domain_preds.eq(domain_y.view_as(domain_preds)).sum().item() / len(domain_preds)

        label_preds = label_preds.argmax(dim=1, keepdim=True)
        label_acc += label_preds.eq(label_y.view_as(label_preds)).sum().item() / len(label_preds)

        if opt.verbose and i > 0 and i % int(n_batches / 10) == 0:
            print('.', flush=True, end='')

        if i >= n_batches:
            break

    domain_loss = total_domain_loss / n_batches
    domain_acc = domain_acc / n_batches
    label_loss = total_label_loss / n_batches
    label_acc = label_acc / n_batches

    return domain_loss, domain_acc, label_loss, label_acc

def test(test_loader, model, criterion, device):
    model.eval()
    model = model.to(device)

    running_loss = 0.
    running_acc = 0.

    with torch.no_grad():
        groundtruth = []
        prediction = []
        

        for i, test_data in enumerate(test_loader):
            #print(test_data)
            acoustic_features =[{"input_values": test_data["input_values"][0][0].to(device)}]
            #labels = torch.tensor(float(test_data["labels"]))
            labels = torch.tensor([test_data["labels"]])
            #acoustic_features, a_labels = test_data

            
                    
            #acoustic_features = [{"input_values":acoustic_features[0][0]}]

            #acoustic_features = acoustic_features.cuda()
            labels = labels.to(device)
            #print(labels)
            
            # final_features = Wav2Vec2Processor.pad(
            #     acoustic_features,
            #     padding=True,
            #     return_tensors="pt",
            # )
            logits = model(test_data['input_values'].squeeze(1).to(device)).logits 
            
            predictions = torch.argmax(logits, dim=-1).detach().cpu()
            
            # predictions = torch.tensor(float(predictions)).unsqueeze(0)
            # print(predictions)
            loss = criterion(logits, labels)

            running_loss += loss.item()

            groundtruth.append(labels.tolist())
            predictions = predictions.argmax(dim=-1, keepdim=True)
            prediction.append(predictions.view_as(labels).tolist())

        test_loss = running_loss / len(test_loader)

        groundtruth = list(itertools.chain.from_iterable(groundtruth))
        prediction = list(itertools.chain.from_iterable(prediction))

        test_acc = accuracy_score(prediction, groundtruth)
        test_uar = recall_score(prediction, groundtruth, average='macro')

        return test_loss, test_acc, test_uar

In [ ]:
import itertools
from sklearn.metrics import accuracy_score, recall_score
batch_size = 8
epochs_num =  10
learning_rate = 1e-3
half_batch = batch_size // 2
batch_size = half_batch

def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

# Dataloaders
test_loader = NSCTest

# Model, optimizer and loss function
# checkpoint = torch.load(os.path.join(	'checkpoints/bl', opt.source_domain, 'model.pth.tar'),
#                     map_location=device)

emotion_recognizer = DANN.class_classifier
# emotion_recognizer.load_state_dict(checkpoint['emotion_recognizer'])
for param in emotion_recognizer.parameters():
    param.requires_grad = True
emotion_recognizer.to(device)

discriminator = DANN.domain_classifier
discriminator.apply(init_weights)
for param in discriminator.parameters():
    param.requires_grad = True
discriminator.to(device)

optimizer = torch.optim.Adam(	list(emotion_recognizer.parameters())
                +list(discriminator.parameters()),
                lr=learning_rate)

criterion = torch.nn.CrossEntropyLoss()

best_acc = 0.
best_acc_std = 0.

best_uar = 0.
best_uar_std = 0.

# Train and validate
for epoch in tqdm(range(epochs_num)):
    print('epoch: {}/{}'.format(epoch + 1, epochs_num))

    batch_iterator, n_batches = get_batch_iterator()
    
    #print(list(batch_iterator))

    domain_loss, domain_acc, train_loss, train_acc = train(list(batch_iterator), n_batches,
                              emotion_recognizer, discriminator,
                              optimizer, criterion, device)
    test_loss, test_acc, test_uar = test(NSCTest, emotion_recognizer, criterion, device)

#     acc_list = []
#     uar_list = []

#     _, acc, uar = test(test_loader, emotion_recognizer, criterion, device)
#     acc_list.append(acc)
#     uar_list.append(uar)

#     acc_list = np.array(acc_list)
#     uar_list = np.array(uar_list)

#     acc_std = np.std(acc_list)
#     uar_std = np.std(uar_list)


    print(	'domain_loss: {0:.5f}'.format(domain_loss),
        'domain_acc: {0:.3f}'.format(domain_acc),
        'train_loss: {0:.5f}'.format(train_loss),
        'train_acc: {0:.3f}'.format(train_acc),
        'test_loss: {0:.5f}'.format(test_loss),
        'test_acc: {0:.3f}'.format(test_acc),
        'test_uar: {0:.3f}'.format(test_uar))

    #os.makedirs(os.path.join(opt.logger_path, opt.source_domain), exist_ok=True)

    model_file_name = 'checkpoint.pth.tar'
    state = {	'epoch': epoch+1, 'emotion_recognizer': emotion_recognizer.state_dict(),
        'discriminator' : discriminator.state_dict()}
    torch.save(state, model_file_name)

    if test_acc > best_acc and epoch >= 3:
        model_file_name = 'model.pth.tar'
        torch.save(state, model_file_name)

        best_acc = test_acc
        #best_acc_std = acc_std

    if test_uar > best_uar and epoch >= 3:
        best_uar = test_uar
        #best_uar_std = uar_std

print(best_acc, best_uar)

## trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset['train'],
    eval_dataset=val_dataset['train'],
)

In [ ]:
trainer.train()

In [ ]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)

In [ ]:
input

In [ ]:
target

In [ ]:
input = torch.randn(3, 5, requires_grad=True)

In [ ]:
input

In [ ]:
target = torch.randn(3, 5).softmax(dim=1)

In [ ]:
target

In [ ]:
output = loss(input, target)

In [53]:
a,b = get_batch_iterator()

In [58]:
b

1